<a href="https://colab.research.google.com/github/hamid-r-h/movies_recommender/blob/main/content_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import nltk
nltk.download('popular')






[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [3]:
import csv
import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer


In [4]:
movie_meta_data = pd.read_csv('drive/MyDrive/project/files/movies_metadata.csv')
keywords = pd.read_csv('drive/MyDrive/project/files/keywords.csv')
credits = pd.read_csv('drive/MyDrive/project/files/credits.csv')


<ipython-input-4-42047b33640f>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_meta_data = pd.read_csv('drive/MyDrive/project/files/movies_metadata.csv')


In [5]:
movie_meta_data = movie_meta_data.drop([19730, 29503, 35587])

movies_data = movie_meta_data.drop(['adult','homepage','video','runtime','revenue','release_date','imdb_id','budget','poster_path','original_title',
                             'original_language','belongs_to_collection','production_countries'],axis=1)


keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
movies_data['id'] = movies_data['id'].astype('int')

df_k_c = keywords.merge(credits,on="id",how="left")
movies_data = df_k_c.merge(movies_data,on="id",how="left")
movies_data

,id,keywords,cast,crew,genres,overview,popularity,production_companies,spoken_languages,status,tagline,title,vote_average,vote_count
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46623,439050,"[{'id': 10703, 'name': 'tragic love'}]","[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...","[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",Rising and falling between a man and woman.,0.072051,[],"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,4.0,1.0
46624,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,...","[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...","[{'id': 18, 'name': 'Drama'}]",An artist struggles to finish his work while a...,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,9.0,3.0
46625,67758,[],"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...","[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...","When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,3.8,6.0
46626,227506,[],"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",[],"In a small town live two brothers, one a minis...",0.003503,"[{'name': 'Yermoliev', 'id': 88753}]",[],Released,NaN,Satan Triumphant,0.0,0.0


In [6]:
movies_data['genres'] = movies_data['genres'].fillna('[]').apply(lambda x: [i['name'] for i in eval(x)])
movies_data['genres'] = movies_data['genres'].apply(lambda x: ' '.join([i.replace(",", "") for i in x]))
movies_data['production_companies'] = movies_data['production_companies'].fillna('[]').apply(lambda x: [i['name'] for i in eval(x)] if isinstance(eval(x), list) else [])
movies_data['production_companies'] = movies_data['production_companies'].apply(lambda x: ' '.join([i.replace(",", "") for i in x]))
movies_data['keywords'] = movies_data['keywords'].fillna('[]').apply(lambda x: [i['name'] for i in eval(x)] if isinstance(eval(x), list) else [])
movies_data['keywords'] = movies_data['keywords'].apply(lambda x: ' '.join([i.replace(",", "") for i in x]))
movies_data['cast'] = movies_data['cast'].fillna('[]').apply(lambda x: [i['name'] for i in eval(x)] if isinstance(eval(x), list) else [])
movies_data['cast'] = movies_data['cast'].apply(lambda x: ' '.join([i.replace(",", "") for i in x]))
movies_data['crew'] = movies_data['crew'].fillna('[]').apply(lambda x: [i['name'] for i in eval(x) if i['job'] == 'Director'] if isinstance(eval(x), list) else [])
movies_data['crew'] = movies_data['crew'].apply(lambda x: ' '.join([i.replace(",", "") for i in x]))
movies_data['spoken_languages'] = movies_data['spoken_languages'].fillna('[]').apply(lambda x: [i['name'] for i in eval(x)] if isinstance(eval(x), list) else [])
movies_data['spoken_languages'] = movies_data['spoken_languages'].apply(lambda x: ' '.join([i.replace(",", "") for i in x]))
movies_data['overview'] = movies_data['overview'].fillna('')
movies_data['tagline'] = movies_data['tagline'].fillna('')

movies_data

,id,keywords,cast,crew,genres,overview,popularity,production_companies,spoken_languages,status,tagline,title,vote_average,vote_count
0,862,jealousy toy boy friendship friends rivalry bo...,Tom Hanks Tim Allen Don Rickles Jim Varney Wal...,John Lasseter,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,English,Released,,Toy Story,7.7,5415.0
1,8844,board game disappearance based on children's b...,Robin Williams Jonathan Hyde Kirsten Dunst Bra...,Joe Johnston,Adventure Fantasy Family,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures Teitler Film Interscope Commu...,English Français,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,15602,fishing best friend duringcreditsstinger old men,Walter Matthau Jack Lemmon Ann-Margret Sophia ...,Howard Deutch,Romance Comedy,A family wedding reignites the ancient feud be...,11.7129,Warner Bros. Lancaster Gate,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,31357,based on novel interracial relationship single...,Whitney Houston Angela Bassett Loretta Devine ...,Forest Whitaker,Comedy Drama Romance,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,11862,baby midlife crisis confidence aging daughter ...,Steve Martin Diane Keaton Martin Short Kimberl...,Charles Shyer,Comedy,Just when George Banks has recovered from his ...,8.387519,Sandollar Productions Touchstone Pictures,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46623,439050,tragic love,Leila Hatami Kourosh Tahami Elham Korda,Hamid Nematollah,Drama Family,Rising and falling between a man and woman.,0.072051,,فارسی,Released,Rising and falling between a man and woman,Subdue,4.0,1.0
46624,111109,artist play pinoy,Angel Aquino Perry Dizon Hazel Orencio Joel To...,Lav Diaz,Drama,An artist struggles to finish his work while a...,0.178241,Sine Olivia,,Released,,Century of Birthing,9.0,3.0
46625,67758,,Erika Eleniak Adam Baldwin Julie du Page James...,Mark L. Lester,Action Drama Thriller,"When one of her hits goes wrong, a professiona...",0.903007,American World Pictures,English,Released,A deadly game of wits.,Betrayal,3.8,6.0
46626,227506,,Iwan Mosschuchin Nathalie Lissenko Pavel Pavlo...,Yakov Protazanov,,"In a small town live two brothers, one a minis...",0.003503,Yermoliev,,Released,,Satan Triumphant,0.0,0.0


In [7]:
movies_des=pd.DataFrame()
movies_des['title']=movies_data['title']
movies_des['vote_average']=movies_data['vote_average']
movies_des['vote_count']=movies_data['vote_count']
movies_des['data']=movies_data['genres'] +' '+movies_data['overview']+' '+movies_data['production_companies'] + ' ' +movies_data['title'] +' '+ movies_data['tagline']+' '+movies_data['crew'] +' '+ movies_data['keywords']
movies_des['data']=movies_des['data'].str.lower()
movies_des.drop_duplicates(inplace=True)
movies_des=movies_des.dropna()
pst = PorterStemmer()
movies_des['data'] = movies_des['data'].apply(word_tokenize)
movies_des['data'] = movies_des['data'].apply(lambda x: ' '.join([pst.stem(y) for y in x]))
movies_des=movies_des.reset_index()
movies_des=movies_des.drop("index" , axis=1)
movies_des

,title,vote_average,vote_count,data
0,Toy Story,7.7,5415.0,"anim comedi famili led by woodi , andi 's toy ..."
1,Jumanji,6.9,2413.0,adventur fantasi famili when sibl judi and pet...
2,Grumpier Old Men,6.5,92.0,romanc comedi a famili wed reignit the ancient...
3,Waiting to Exhale,6.1,34.0,"comedi drama romanc cheat on , mistreat and st..."
4,Father of the Bride Part II,5.7,173.0,comedi just when georg bank ha recov from hi d...
...,...,...,...,...
45427,Subdue,4.0,1.0,drama famili rise and fall between a man and w...
45428,Century of Birthing,9.0,3.0,drama an artist struggl to finish hi work whil...
45429,Betrayal,3.8,6.0,action drama thriller when one of her hit goe ...
45430,Satan Triumphant,0.0,0.0,"in a small town live two brother , one a minis..."


In [8]:
class movies_recommender :

    def __init__(self,movies_des):
        self.movies_des = movies_des
    def vectorize(self):
        tfid = TfidfVectorizer(analyzer='word',stop_words='english')
        tfid_vector = tfid.fit_transform(movies_des['data'])
        return tfid_vector
    def content_base_recommender(self,tfid_vector,title) :
        indx=movies_des[movies_des["title"]==title].index
        sim_vector = linear_kernel(tfid_vector,tfid_vector[indx[0]])
        same_rate = sorted(list(enumerate(sim_vector)), key=lambda x: x[1], reverse=True)
        same_rate = same_rate[1:30]
        movie_indices = [i[0] for i in same_rate]
        result=movies_des.iloc[movie_indices]
        result.sort_values(by=["vote_average","vote_count"],ascending=[True,True])
        result=result.drop("data",axis=1)
        return result[:10]

In [9]:
movies_rec=movies_recommender(movies_des=movies_des)
tfid_vector=movies_rec.vectorize()
result =movies_rec.content_base_recommender(tfid_vector,"The Dark Knight")
result

,title,vote_average,vote_count
18247,The Dark Knight Rises,7.6,9263.0
10121,Batman Begins,7.5,7511.0
9230,Batman Beyond: Return of the Joker,7.5,152.0
20225,"Batman: The Dark Knight Returns, Part 2",7.9,426.0
15507,Batman: Under the Red Hood,7.6,459.0
19785,"Batman: The Dark Knight Returns, Part 1",7.7,410.0
40944,LEGO DC Comics Super Heroes: Batman: Be-Leaguered,6.1,20.0
3094,Batman: Mask of the Phantasm,7.4,218.0
585,Batman,7.0,2145.0
39598,Batman: The Killing Joke,6.2,485.0
